In [4]:
import yfinance as yf
import os
import pandas as pd
from datetime import datetime

current_year = str(datetime.now().year)

### Télécharger les données historiques du BRENT/USD (BZ=F)
ticker_brent = "BZ=F"
df_brent_usd = yf.download(ticker_brent, start=("2007-01-01"), end=current_year+"-12-31", auto_adjust=False)

df_brent_usd.columns = df_brent_usd.columns.droplevel(1)
df_brent_usd.reset_index(inplace=True)

df_brent_usd["Brent_USD_liter"] = df_brent_usd["Close"].round(3)
df_brent_usd = df_brent_usd.drop(columns=["Adj Close", "Volume", "High", "Low", "Open", "Close"])

# Garder la première valeur de chaque mois
df_brent_usd_monthly = (
    df_brent_usd
        .groupby(df_brent_usd["Date"].dt.to_period("M"))
        .first()
        .reset_index(drop=True)
)

# Mettre toutes les dates au premier du mois
df_brent_usd_monthly["Date_monthly"] = df_brent_usd_monthly["Date"].dt.to_period("M").dt.to_timestamp()
df_brent_usd_monthly = df_brent_usd_monthly.drop(columns=["Date"])

print(df_brent_usd_monthly.head())




#### GET EUR USD by CSV
df_eur_usd_monthly_avg = pd.read_csv(
    "csv/eur_usd_monthly.csv",
    parse_dates=["Date_monthly"]
)

print("df_eur_usd_monthly_avg\n", df_eur_usd_monthly_avg)


# Merge monthly EUR/USD average with brent_usd_monthly
df_merged = pd.merge(df_brent_usd_monthly, df_eur_usd_monthly_avg, on='Date_monthly', how='left')
df_merged['brent_EUR_liter'] = (df_merged['Brent_USD_liter']/ df_merged['EUR_USD_monthly_avg']).round(5)
df_merged = df_merged.drop(columns=["Brent_USD_liter", "EUR_USD_monthly_avg"])
print('df_merged\n', df_merged)

os.makedirs("csv", exist_ok=True)
df_merged.to_csv("csv/brent_eur_liter_monthly.csv", index=False)

[*********************100%***********************]  1 of 1 completed

Price  Brent_USD_liter Date_monthly
0                75.74   2007-07-01
1                75.35   2007-08-01
2                73.92   2007-09-01
3                77.64   2007-10-01
4                89.72   2007-11-01
df_eur_usd_monthly_avg
      EUR_USD_monthly_avg Date_monthly
0                 1.3219   2007-01-01
1                 1.3019   2007-02-01
2                 1.3170   2007-03-01
3                 1.3370   2007-04-01
4                 1.3601   2007-05-01
..                   ...          ...
225               1.1736   2025-10-01
226               1.1528   2025-11-01
227               1.1601   2025-12-01
228               1.1750   2026-01-01
229               1.1849   2026-02-01

[230 rows x 2 columns]
df_merged
     Date_monthly  brent_EUR_liter
0     2007-07-01         55.54822
1     2007-08-01         55.09249
2     2007-09-01         54.29705
3     2007-10-01         54.53779
4     2007-11-01         62.18895
..           ...              ...
219   2025-10-01         55.683